In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from snmfem.experiments import perform_simulations, load_samples, print_results, load_data, run_experiment
import re




In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)


# Running NMF

## Inputs

In [ ]:
dataset = "dataset_EDXS_small.json"
samples, k, shape_2d = load_samples(dataset)
n_samples = 3
samples = samples[:n_samples]



In [ ]:
default_params = {
    "n_components" : k,
    "tol" : 1e-5,
    "max_iter" : 1000,
    "init" : "nndsvda",
    "random_state" : 1,
    "verbose" : 0
    }

params_snmf = {
    "force_simplex" : True,
    "skip_G" : False,
}

params_evalution = {
    "u" : True,
}

# All parameters are contained here
exp_list = [ 
    {"name": "snmfem", "method": "NMF", "params": {**default_params, **params_snmf}},
    {"name": "snmfem smooth 1", "method": "SmoothNMF", "params": {**default_params, **params_snmf, "lambda_L" : 1.0, "shape_2d" : shape_2d}},
    {"name": "snmfem smooth 10", "method": "SmoothNMF", "params": {**default_params, **params_snmf, "lambda_L" : 10.0, "shape_2d" : shape_2d}},
    {"name": "snmfem smooth 100", "method": "SmoothNMF", "params": {**default_params, **params_snmf, "lambda_L" : 100.0, "shape_2d" : shape_2d}},
    {"name": "scikit KL", "method": "SKNMF", "params": {**default_params, "beta_loss" : "kullback-leibler", "solver":"mu"}},
    {"name": "scikit L2", "method": "SKNMF", "params": {**default_params, }},
]

In [ ]:
metrics = perform_simulations(samples, exp_list, params_evalution)

In [ ]:
metrics.shape

In [ ]:

print_results(exp_list, metrics)

# Run a single experiment and plotting results

In [ ]:
sample_num = 0

s = samples[sample_num]
Xflat, true_spectra, true_maps, G = load_data(s)

GPs = []
metrics = []
As = []

for exp in exp_list : 
    metric, (GP, A), _  = run_experiment(Xflat, true_spectra, true_maps, G, exp, params_evalution)
    GPs.append(GP)
    As.append(A)
    metrics.append(metric)
metrics = np.array(metrics)

In [ ]:
metrics.shape

In [ ]:
# Ploting parameters
fontsize = 15
aspect_ratio = 3/4
scale = 20



In [ ]:
n_exp = len(exp_list)
fig, axes = plt.subplots(n_exp,k,figsize = (scale,scale/k * n_exp * aspect_ratio))
for i, exp in enumerate(exp_list): 
    for j in range(k):
        axes[i,j].plot(true_spectra[j],'bo',label='truth',linewidth=4)
        axes[i,j].plot(GPs[i][:,j] ,'r-',label='reconstructed',markersize=3.5)
        axes[i,j].set_title("{:.2f} deg".format(metrics[i,0,j]))

cols = ['Phase {}'.format(col) for col in range(k)]
rows = ['{}'.format(exp["name"]) for exp in exp_list]

for ax, col in zip(axes[0], cols):
    ax.set_title(col, fontsize=fontsize)

for ax, row in zip(axes[:,0], rows):
    ax.set_ylabel(row, rotation=90, fontsize=fontsize)

fig.tight_layout()


In [ ]:
n_exp = len(exp_list)
fig, axes = plt.subplots(n_exp+1,k,figsize = (scale,scale/k * (n_exp+1) * aspect_ratio))
for j in range(k):
    axes[0,j].imshow(true_maps[j].reshape(*shape_2d))
    for i, exp in enumerate(exp_list): 
        axes[i+1,j].imshow(As[i][j].reshape(*shape_2d))
        axes[i+1,j].set_title("Mse: {:.2f}".format(metrics[i,1,j]))
    
cols = ['MAP {}'.format(col) for col in range(k)]
rows = ["Ground truth"] + ['{}'.format(exp["name"]) for exp in exp_list]

for ax, col in zip(axes[0], cols):
    ax.set_title(col, fontsize=fontsize)

for ax, row in zip(axes[:,0], rows):
    ax.set_ylabel(row, rotation=90, fontsize=fontsize)

fig.tight_layout()

# Quick and dirty way to look at the Loss
Not updated... We need to uniform the way we store the loss for all methods.


In [ ]:
warnings.simplefilter(action='ignore', category=FutureWarning)

force_simplex = True
skip_G = False
alpha = 1.0
index = 2
lambda_L = 10.0
nx,ny = data_dict["weights_parameters"]["shape_2D"]
shape_2D = nx,ny
k = default_params["n_components"]
Xflat, true_spectra, true_maps, G = load_data(samples[index])
snmfem = run_snmfem_NMF(Xflat, true_spectra, true_maps, G,default_params,force_simplex=force_simplex,skip_G = skip_G)
smooth_snmfem = run_snmfem_SmoothNMF(Xflat,true_spectra,true_maps,G,default_params,force_simplex=force_simplex,skip_G=skip_G,lambda_L=lambda_L,shape_2D = shape_2D)




In [ ]:
%matplotlib inline
plt.subplots(1,snmfem[-1].T.shape[0],figsize=(50, 12))
plt.rcParams.update({'font.size': 22})
for i,elt in enumerate(snmfem[-1].T) : 
    plt.subplot(1,snmfem[-1].T.shape[0],i+1)
    list_title = ["full loss", "base_loss", "assymetric reg", "rel_P", "rel_A"]
    plt.title(list_title[i])
    plt.plot( elt,'r-',label='reconstructed',markersize=3.5)
    plt.xlabel("number of iterations")

In [ ]:
%matplotlib inline
plt.subplots(1,smooth_snmfem[-1].T.shape[0],figsize=(50, 12))
plt.rcParams.update({'font.size': 22})
for i,elt in enumerate(smooth_snmfem[-1].T) : 
    plt.subplot(1,smooth_snmfem[-1].T.shape[0],i+1)
    list_title = ["full loss", "base_loss", "assymetric reg","laplacian reg", "rel_P", "rel_A"]
    plt.title(list_title[i])
    plt.plot( elt,'r-',label='reconstructed',markersize=3.5)
    plt.xlabel("number of iterations")